In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [8]:
# Initialize MLflow
mlflow.set_experiment("Deployment model")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/08/23 23:59:34 INFO mlflow.tracking.fluent: Experiment with name 'Deployment model' does not exist. Creating a new experiment.


2025/08/23 23:59:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/23 23:59:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/183923839857221422/runs/40eb53c855744e72bacd272fb4c46d4d
🧪 View experiment at: http://localhost:5000/#/experiments/183923839857221422


2025/08/24 00:00:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 00:00:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/183923839857221422/runs/9334c431664e49f7adf1a355e30f4b1d
🧪 View experiment at: http://localhost:5000/#/experiments/183923839857221422


2025/08/24 00:00:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 00:00:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/183923839857221422/runs/7968908240de460887757769d283f8c6
🧪 View experiment at: http://localhost:5000/#/experiments/183923839857221422


2025/08/24 00:00:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/24 00:00:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/183923839857221422/runs/2dee448111fc4db0afcb7fa4ea0e8938
🧪 View experiment at: http://localhost:5000/#/experiments/183923839857221422


### Register the Model

In [13]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/08/24 00:03:56 WARNING mlflow.tracking._model_registry.fluent: Run with id 7968908240de460887757769d283f8c6 has no artifacts at artifact path 'model', registering model based on models:/m-353009bcc1404f5abd3c9e266d876050 instead
2025/08/24 00:03:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 2


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/183923839857221422/runs/7968908240de460887757769d283f8c6
🧪 View experiment at: http://localhost:5000/#/experiments/183923839857221422


Created version '2' of model 'XGB-Smote'.


### Load the Model

In [12]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production server

In [11]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

versions = client.search_model_versions("name='XGB-Smote'")
for v in versions:
    print(f"Name: {v.name}, Version: {v.version}, Aliases: {v.aliases}, Status: {v.status}")


Name: XGB-Smote, Version: 1, Aliases: [], Status: READY


In [ ]:
'''
# ignore below code

model_version = 2
prod_model_uri = f"models:/{production_model_name}@productionserver2"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]
'''

'\n# ignore below code\n\nmodel_version = 2\nprod_model_uri = f"models:/{production_model_name}@productionserver2"\n\nloaded_model = mlflow.xgboost.load_model(prod_model_uri)\ny_pred = loaded_model.predict(X_test)\ny_pred[:4]\n'